IMPORT ALGORITMA

In [72]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

#Modeling
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedKFold
import lightgbm as lgbm
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score      
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.linear_model import ElasticNet, BayesianRidge



from catboost import CatBoostRegressor

#from sklego.linear_model import LADRegression
from xgboost import XGBRegressor
from scipy.stats import yeojohnson
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from mlxtend.regressor import StackingCVRegressor
from sklearn.preprocessing import PolynomialFeatures

import warnings
warnings.filterwarnings('ignore')

In [73]:
import pandas as pd

# Definisikan nama file
filename = "booking_dataset_2024_05_27.csv"

# Membaca dataset dari file CSV
raw_data = pd.read_csv(filename)

# Menampilkan 5 baris pertama dari dataset
raw_data.head()

,booking_id,booking_status,booking_check_in,booking_check_out,booking_guest_number,booking_earned,booking_currency,booking_received_timestamp,listing_id,booking_earned_in_idr,...,workspace,pool,parking,gym,kitchen,property_id1,area_id,area_name,area_distance_to_airport,airport_pickup_price_idr
0,+44 7737 446047,1,2019-05-16,2019-05-18,NaN,460000.0,1,2019-05-15 16:26,400290105,460000.0,...,1.0,0.0,1.0,0.0,0.0,PR0314,A0002,Uluwatu,1 hour,325000.0
1,1006390676,1,2018-10-07,2018-10-10,2.0,1.0,1,2018-10-06 16:51,28646196,1.0,...,1.0,0.0,1.0,0.0,1.0,PR0494,A0002,Uluwatu,1 hour,325000.0
2,1011500224,1,2018-12-30,2019-01-01,NaN,1300000.0,1,2018-12-08 13:37,3736998,1300000.0,...,1.0,0.0,0.0,0.0,1.0,PR0045,A0015,Seminyak,45 minutes,195000.0
3,1011500224,1,2018-12-30,2019-01-01,NaN,1300000.0,1,2018-12-08 13:37,3736998,1300000.0,...,1.0,0.0,0.0,0.0,1.0,PR0045,A0015,Seminyak,45 minutes,195000.0
4,1019874684,1,2018-07-07,2018-07-08,2.0,441394.0,1,2018-07-07 00:00,23241329,441394.0,...,1.0,0.0,1.0,0.0,1.0,PR0294,A0043,Ngupasan,NaN,NaN


checking data nan


In [74]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56641 entries, 0 to 56640
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   booking_id                  56641 non-null  object 
 1   booking_status              56641 non-null  int64  
 2   booking_check_in            56641 non-null  object 
 3   booking_check_out           56641 non-null  object 
 4   booking_guest_number        8945 non-null   float64
 5   booking_earned              56641 non-null  float64
 6   booking_currency            56641 non-null  int64  
 7   booking_received_timestamp  30261 non-null  object 
 8   listing_id                  56641 non-null  object 
 9   booking_earned_in_idr       56641 non-null  float64
 10  length_of_stay              56641 non-null  int64  
 11  booking_window              30261 non-null  float64
 12  average_daily_rate          56641 non-null  float64
 13  unit_id                     566

In [75]:
raw_data.describe()

,booking_status,booking_guest_number,booking_earned,booking_currency,booking_earned_in_idr,length_of_stay,booking_window,average_daily_rate,status,bedroom,...,wifi,tv,cable_tv,ac,workspace,pool,parking,gym,kitchen,airport_pickup_price_idr
count,56641.000000,8945.000000,5.664100e+04,56641.000000,5.664100e+04,56641.000000,30261.000000,5.664100e+04,56194.000000,56128.000000,...,41924.000000,41924.000000,41924.000000,41924.000000,41924.000000,41924.000000,41924.000000,41924.000000,41924.000000,47921.000000
mean,1.064123,2.622582,1.571724e+06,1.233718,2.678077e+06,2.840698,23.076336,9.221712e+05,1.725843,1.456100,...,0.989457,0.576114,0.214293,0.991723,0.936480,0.821224,0.947142,0.005963,0.321343,325700.527952
std,0.281902,2.280523,3.035599e+06,0.427556,4.572227e+06,2.984960,36.982980,1.016009e+06,0.446093,0.970036,...,0.102137,0.494179,0.410336,0.090601,0.243898,0.383169,0.223752,0.076992,0.466998,39046.514382
min,1.000000,0.000000,0.000000e+00,1.000000,0.000000e+00,-30.000000,0.000000,-4.938270e+04,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,195000.000000
25%,1.000000,2.000000,6.390000e+02,1.000000,6.984000e+05,1.000000,1.000000,3.576110e+05,1.000000,1.000000,...,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,325000.000000
50%,1.000000,2.000000,7.992800e+05,1.000000,1.376080e+06,2.000000,6.000000,5.599180e+05,2.000000,1.000000,...,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,325000.000000
75%,1.000000,2.000000,1.781274e+06,1.000000,2.793903e+06,3.000000,31.000000,1.164000e+06,2.000000,1.000000,...,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,325000.000000
max,6.000000,30.000000,9.429215e+07,3.000000,1.303185e+08,349.000000,383.000000,2.129842e+07,2.000000,8.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,585000.000000


In [76]:
raw_data.isnull().sum()

booking_id                        0
booking_status                    0
booking_check_in                  0
booking_check_out                 0
booking_guest_number          47696
booking_earned                    0
booking_currency                  0
booking_received_timestamp    26380
listing_id                        0
booking_earned_in_idr             0
length_of_stay                    0
booking_window                26380
average_daily_rate                0
unit_id                          14
status                          447
bedroom                         513
bathroom                      11575
beds                          11575
capacity                      15526
property_id                     447
avg_daily_rate_per_unit           0
wifi                          14717
tv                            14717
cable_tv                      14717
ac                            14717
workspace                     14717
pool                          14717
parking                     

In [77]:

raw_data['booking_check_in'] = pd.to_datetime(raw_data['booking_check_in'])
raw_data['booking_check_out'] = pd.to_datetime(raw_data['booking_check_out'])

def convert_to_minutes(time_str):
    if pd.isna(time_str):
        return None
    if 'hour' in time_str:
        hours = int(time_str.split()[0])
        return hours * 60
    elif 'minute' in time_str:
        minutes = int(time_str.split()[0])
        return minutes
    else:
        return None

raw_data['area_distance_to_airport'] = raw_data['area_distance_to_airport'].apply(convert_to_minutes)

raw_data['booking_check_in_date'] = raw_data['booking_check_in'].dt.day
raw_data['booking_check_in_month'] = raw_data['booking_check_in'].dt.month
raw_data['booking_check_in_year'] = raw_data['booking_check_in'].dt.year

raw_data['booking_check_out_date'] = raw_data['booking_check_out'].dt.day
raw_data['booking_check_out_month'] = raw_data['booking_check_out'].dt.month
raw_data['booking_check_out_year'] = raw_data['booking_check_out'].dt.year



In [78]:
raw_data = raw_data.drop(['booking_id','booking_window','booking_check_in','booking_check_out','booking_received_timestamp',
'listing_id','property_id1',], axis = 1)


In [79]:
raw_data.describe()

,booking_status,booking_guest_number,booking_earned,booking_currency,booking_earned_in_idr,length_of_stay,average_daily_rate,status,bedroom,bathroom,...,gym,kitchen,area_distance_to_airport,airport_pickup_price_idr,booking_check_in_date,booking_check_in_month,booking_check_in_year,booking_check_out_date,booking_check_out_month,booking_check_out_year
count,56641.000000,8945.000000,5.664100e+04,56641.000000,5.664100e+04,56641.000000,5.664100e+04,56194.000000,56128.000000,45066.000000,...,41924.000000,41924.000000,55172.000000,47921.000000,56641.000000,56641.000000,56641.000000,56641.000000,56641.000000,56641.000000
mean,1.064123,2.622582,1.571724e+06,1.233718,2.678077e+06,2.840698,9.221712e+05,1.725843,1.456100,1.084920,...,0.005963,0.321343,53.895273,325700.527952,15.898254,6.642150,2017.929027,15.886301,6.627408,2017.938048
std,0.281902,2.280523,3.035599e+06,0.427556,4.572227e+06,2.984960,1.016009e+06,0.446093,0.970036,0.734498,...,0.076992,0.466998,12.321309,39046.514382,8.796246,3.003752,1.108604,8.841998,3.006187,1.106714
min,1.000000,0.000000,0.000000e+00,1.000000,0.000000e+00,-30.000000,-4.938270e+04,1.000000,1.000000,0.000000,...,0.000000,0.000000,30.000000,195000.000000,1.000000,1.000000,2014.000000,1.000000,1.000000,2014.000000
25%,1.000000,2.000000,6.390000e+02,1.000000,6.984000e+05,1.000000,3.576110e+05,1.000000,1.000000,1.000000,...,0.000000,0.000000,45.000000,325000.000000,8.000000,4.000000,2017.000000,8.000000,4.000000,2017.000000
50%,1.000000,2.000000,7.992800e+05,1.000000,1.376080e+06,2.000000,5.599180e+05,2.000000,1.000000,1.000000,...,0.000000,0.000000,60.000000,325000.000000,16.000000,7.000000,2018.000000,16.000000,7.000000,2018.000000
75%,1.000000,2.000000,1.781274e+06,1.000000,2.793903e+06,3.000000,1.164000e+06,2.000000,1.000000,1.000000,...,0.000000,1.000000,60.000000,325000.000000,23.000000,9.000000,2019.000000,24.000000,9.000000,2019.000000
max,6.000000,30.000000,9.429215e+07,3.000000,1.303185e+08,349.000000,2.129842e+07,2.000000,8.000000,8.000000,...,1.000000,1.000000,120.000000,585000.000000,31.000000,12.000000,2019.000000,31.000000,12.000000,2019.000000


In [80]:
raw_data.isnull().sum()

booking_status                  0
booking_guest_number        47696
booking_earned                  0
booking_currency                0
booking_earned_in_idr           0
length_of_stay                  0
average_daily_rate              0
unit_id                        14
status                        447
bedroom                       513
bathroom                    11575
beds                        11575
capacity                    15526
property_id                   447
avg_daily_rate_per_unit         0
wifi                        14717
tv                          14717
cable_tv                    14717
ac                          14717
workspace                   14717
pool                        14717
parking                     14717
gym                         14717
kitchen                     14717
area_id                       578
area_name                     578
area_distance_to_airport     1469
airport_pickup_price_idr     8720
booking_check_in_date           0
booking_check_

In [81]:
from sklearn.preprocessing import LabelEncoder

# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()

# Menggunakan LabelEncoder untuk kolom unit_id
raw_data['unit_id'] = label_encoder.fit_transform(raw_data['unit_id'])

# Menggunakan LabelEncoder untuk kolom property_id
raw_data['property_id'] = label_encoder.fit_transform(raw_data['property_id'])

# Menggunakan LabelEncoder untuk kolom area_id
raw_data['area_id'] = label_encoder.fit_transform(raw_data['area_id'])

# Menggunakan LabelEncoder untuk kolom area_name
raw_data['area_name'] = label_encoder.fit_transform(raw_data['area_name'])

# Menampilkan hasil konversi
print(raw_data[['unit_id', 'property_id', 'area_id', 'area_name']].head())


   unit_id  property_id  area_id  area_name
0      834          248        2         21
1     1240          357        2         21
2      749           37       14         17
3      752           37       14         17
4      341          232       21         12


In [82]:
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# Memilih kolom yang akan digunakan untuk clustering
columns_for_clustering = ['bedroom', 'bathroom', 'beds', 'capacity','wifi','tv','cable_tv','ac','workspace','pool',
                          'booking_guest_number','parking','gym','kitchen', 'avg_daily_rate_per_unit','status']

# Mengisi nilai yang hilang sementara dengan SimpleImputer (mean strategy)
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(raw_data[columns_for_clustering])

# Normalisasi data sebelum clustering
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)

# Menentukan jumlah cluster (misalnya, 5)
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(data_scaled)

# Menambahkan kolom cluster ke DataFrame asli
raw_data['cluster'] = clusters

# Mengisi nilai yang hilang dengan centroid dari masing-masing cluster
for column in columns_for_clustering:
    for cluster in range(5):
        cluster_mean = raw_data[raw_data['cluster'] == cluster][column].mean()
        raw_data.loc[(raw_data[column].isnull()) & (raw_data['cluster'] == cluster), column] = cluster_mean

# Mengecek kembali nilai yang hilang setelah imputasi
print("\nJumlah nilai yang hilang setelah imputasi:\n", raw_data.isnull().sum())

# Menghapus kolom cluster karena sudah tidak diperlukan
raw_data.drop(columns=['cluster'], inplace=True)


Jumlah nilai yang hilang setelah imputasi:
 booking_status                 0
booking_guest_number           0
booking_earned                 0
booking_currency               0
booking_earned_in_idr          0
length_of_stay                 0
average_daily_rate             0
unit_id                        0
status                         0
bedroom                        0
bathroom                       0
beds                           0
capacity                       0
property_id                    0
avg_daily_rate_per_unit        0
wifi                           0
tv                             0
cable_tv                       0
ac                             0
workspace                      0
pool                           0
parking                        0
gym                            0
kitchen                        0
area_id                        0
area_name                      0
area_distance_to_airport    1469
airport_pickup_price_idr    8720
booking_check_in_date          

In [83]:
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# Memilih kolom yang akan digunakan untuk clustering
columns_for_clustering = ['booking_check_in_date','booking_check_in_month','booking_check_in_year',
                          'booking_check_in_year','booking_check_out_month','booking_check_out_year']

# Mengisi nilai yang hilang sementara dengan SimpleImputer (mean strategy)
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(raw_data[columns_for_clustering])

# Normalisasi data sebelum clustering
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)

# Menentukan jumlah cluster (misalnya, 5)
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(data_scaled)

# Menambahkan kolom cluster ke DataFrame asli
raw_data['cluster'] = clusters

# Mengisi nilai yang hilang dengan centroid dari masing-masing cluster
for column in columns_for_clustering:
    for cluster in range(5):
        cluster_mean = raw_data[raw_data['cluster'] == cluster][column].mean()
        raw_data.loc[(raw_data[column].isnull()) & (raw_data['cluster'] == cluster), column] = cluster_mean

# Mengecek kembali nilai yang hilang setelah imputasi
print("\nJumlah nilai yang hilang setelah imputasi:\n", raw_data.isnull().sum())

# Menghapus kolom cluster karena sudah tidak diperlukan
raw_data.drop(columns=['cluster'], inplace=True)


Jumlah nilai yang hilang setelah imputasi:
 booking_status                 0
booking_guest_number           0
booking_earned                 0
booking_currency               0
booking_earned_in_idr          0
length_of_stay                 0
average_daily_rate             0
unit_id                        0
status                         0
bedroom                        0
bathroom                       0
beds                           0
capacity                       0
property_id                    0
avg_daily_rate_per_unit        0
wifi                           0
tv                             0
cable_tv                       0
ac                             0
workspace                      0
pool                           0
parking                        0
gym                            0
kitchen                        0
area_id                        0
area_name                      0
area_distance_to_airport    1469
airport_pickup_price_idr    8720
booking_check_in_date          

In [84]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
columns_to_impute = [ 'airport_pickup_price_idr','area_distance_to_airport']
imputer = IterativeImputer(max_iter=10, random_state=0)

# Memilih subset data untuk kolom-kolom yang akan diimputasi
data_to_impute = raw_data[columns_to_impute]

# Melakukan imputasi
data_imputed = imputer.fit_transform(data_to_impute)

# Mengganti nilai yang telah diimputasi kembali ke dataset asli
raw_data[columns_to_impute] = data_imputed

# Menampilkan hasil imputasi
print("Hasil imputasi untuk kolom area_distance_to_airport dan airport_pickup_price_idr:")
print(raw_data[columns_to_impute].head())
raw_data.info()

Hasil imputasi untuk kolom area_distance_to_airport dan airport_pickup_price_idr:
   airport_pickup_price_idr  area_distance_to_airport
0             325000.000000                 60.000000
1             325000.000000                 60.000000
2             195000.000000                 45.000000
3             195000.000000                 45.000000
4             325761.160221                 53.879866
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56641 entries, 0 to 56640
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   booking_status            56641 non-null  int64  
 1   booking_guest_number      56641 non-null  float64
 2   booking_earned            56641 non-null  float64
 3   booking_currency          56641 non-null  int64  
 4   booking_earned_in_idr     56641 non-null  float64
 5   length_of_stay            56641 non-null  int64  
 6   average_daily_rate        56641 non-null

In [85]:
data = raw_data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56641 entries, 0 to 56640
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   booking_status            56641 non-null  int64  
 1   booking_guest_number      56641 non-null  float64
 2   booking_earned            56641 non-null  float64
 3   booking_currency          56641 non-null  int64  
 4   booking_earned_in_idr     56641 non-null  float64
 5   length_of_stay            56641 non-null  int64  
 6   average_daily_rate        56641 non-null  float64
 7   unit_id                   56641 non-null  int32  
 8   status                    56641 non-null  float64
 9   bedroom                   56641 non-null  float64
 10  bathroom                  56641 non-null  float64
 11  beds                      56641 non-null  float64
 12  capacity                  56641 non-null  float64
 13  property_id               56641 non-null  int32  
 14  avg_da

In [86]:
data.isnull().sum()

booking_status              0
booking_guest_number        0
booking_earned              0
booking_currency            0
booking_earned_in_idr       0
length_of_stay              0
average_daily_rate          0
unit_id                     0
status                      0
bedroom                     0
bathroom                    0
beds                        0
capacity                    0
property_id                 0
avg_daily_rate_per_unit     0
wifi                        0
tv                          0
cable_tv                    0
ac                          0
workspace                   0
pool                        0
parking                     0
gym                         0
kitchen                     0
area_id                     0
area_name                   0
area_distance_to_airport    0
airport_pickup_price_idr    0
booking_check_in_date       0
booking_check_in_month      0
booking_check_in_year       0
booking_check_out_date      0
booking_check_out_month     0
booking_ch

In [87]:
from sklearn.model_selection import train_test_split

# Memisahkan fitur-fitur dan target
X = data.drop(columns=['booking_earned_in_idr'])
y = data['booking_earned_in_idr']

# Memisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Menampilkan dimensi dari data latih dan data uji
print("Dimensi data latih:", X_train.shape, y_train.shape)
print("Dimensi data uji:", X_test.shape, y_test.shape)

Dimensi data latih: (45312, 33) (45312,)
Dimensi data uji: (11329, 33) (11329,)


In [88]:
y = data['booking_earned_in_idr']
x = data.drop('booking_earned_in_idr', axis=1)

In [89]:
# apply transformation since it is skewed as we saw in the eda phase
data['booking_earned_in_idr'] = np.log1p(data['booking_earned_in_idr'])

In [90]:
skew_check = x.apply(lambda x: abs(skew(x)))
skew_check = skew_check.reset_index(name='skew')
skewness = skew_check[skew_check['skew']>0.5]
skewed_feats = skewness['index'][:-1]
skewness

,index,skew
0,booking_status,5.880238
1,booking_guest_number,5.952926
2,booking_earned,7.348172
3,booking_currency,1.329400
4,length_of_stay,33.487546
5,average_daily_rate,3.644372
6,unit_id,0.587965
7,status,1.017404
8,bedroom,2.216139
9,bathroom,4.311759


In [91]:
from sklearn.preprocessing import MinMaxScaler
# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()

# Terapkan normalisasi Min-Max pada fitur numerik
x_scaled = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)

# Gabungkan kembali fitur yang sudah dinormalisasi dengan label
data_normalized = pd.concat([x_scaled, y.reset_index(drop=True)], axis=1)

In [92]:
# apply polynomial since the features are not very linear with the label, and set the degree to 2 to prevent overfitting
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(x)

In [122]:
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import RepeatedKFold
import pandas as pd

class Model:
    def __init__(self, x, y, n_splits, repeated, rdm_seed, model, params):
        self.x = x
        self.y = y
        self.n_splits = n_splits
        self.repeated = repeated
        self.rdm_seed = rdm_seed
        self.model = model
        self.params = params
        self.cv_splitter = None
        self.best_model = None
        self.best_rmse = float('inf')
        self.best_mae = float('inf')  # Initialize best_mae attribute

    def split_data(self):
        cv = RepeatedKFold(n_splits=self.n_splits, n_repeats=self.repeated, random_state=self.rdm_seed)
        for fold, (train_idx, val_idx) in enumerate(cv.split(self.x, self.y)):
            X_train = self.x[train_idx]
            X_val = self.x[val_idx]
            y_train = self.y[train_idx]
            y_val = self.y[val_idx]
            yield X_train, X_val, y_train, y_val

    def fit(self, key):
        mae_per_fold = []
        r2_per_fold = []
        self.cv_splitter = self.split_data()

        for fold, (X_train, X_val, y_train, y_val) in enumerate(self.cv_splitter):
            model = self.model(**self.params)  # Define model outside if-elif block
            if key == 'xgb':
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='mae', early_stopping_rounds=100, verbose=False)
            elif key == 'lgb':
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='mae')
            elif key == 'cat':
                model = CatBoostRegressor(**self.params, loss_function='MAE')
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
            elif key == 'bayesian_ridge':
                model = BayesianRidge(**self.params)
                model.fit(X_train, y_train)
            elif key in ['linear', 'ridge', 'lasso', 'svr']:
                model.fit(X_train, y_train)

            y_val_preds = model.predict(X_val)
            mae = mean_absolute_error(y_val, y_val_preds)
            r2 = r2_score(y_val, y_val_preds)

            print(f"FOLD {fold} | MAE: {mae:.5f} | R2: {r2:.5f}")
            mae_per_fold.append(mae)
            r2_per_fold.append(r2)

            # Select the model with the lowest MAE
            if mae < self.best_mae:
                self.best_mae = mae
                self.best_model = model

        print(f"Average MAE across all folds: {np.mean(mae_per_fold):.5f}")
        print(f"Average R2 across all folds: {np.mean(r2_per_fold):.5f}")

    def predict(self):
        if self.best_model is None:
            raise ValueError("Model has not been trained yet. Call 'fit()' method first.")

        all_mae = []
        all_r2 = []
        y_preds_list = []
        y_val_list = []

        for fold, (X_train, X_val, y_train, y_val) in enumerate(self.split_data()):
            y_preds = self.best_model.predict(X_val)
            mae = mean_absolute_error(y_val, y_preds)
            r2 = r2_score(y_val, y_preds)
            all_mae.append(mae)
            all_r2.append(r2)
            y_preds_list.append(y_preds)
            y_val_list.append(y_val)

            print(f"FOLD {fold} | MAE: {mae:.5f} | R2: {r2:.5f}")

        avg_mae = np.mean(all_mae)
        avg_r2 = np.mean(all_r2)
        print(f'Average MAE Val: {avg_mae:.5f} | Average R2 Val: {avg_r2:.5f}')
        return all_mae, all_r2, y_preds_list, y_val_list

    def feature_importance(self):
        if self.best_model is None:
            raise ValueError("Model has not been trained yet. Call 'fit()' method first.")
        feature_importances = self.best_model.feature_importances_
        n_original_features = len(self.x.columns)
        polynomial_feature_indices = np.arange(n_original_features, self.x.shape[1])
        polynomial_feature_names = [f"Poly_{i}" for i in polynomial_feature_indices]
        all_feature_names = list(self.x.columns) + polynomial_feature_names
        feature_importances_df = pd.data({'feature_weight': feature_importances}, index=all_feature_names)
        feature_importances_df = feature_importances_df.sort_values('feature_weight', ascending=False)
        return feature_importances_df

    def inverse(self, y_preds, y_val):
        y_preds = np.expm1(y_preds)
        y_val = np.expm1(y_val)
        return y_preds, y_val


In [94]:
    def split_data(self):
        cv = RepeatedKFold(n_splits=self.n_splits, n_repeats=self.repeated, random_state=self.rdm_seed)
        for fold, (train_idx, val_idx) in enumerate(cv.split(self.x, self.y)):
            X_train = self.x.iloc[train_idx]
            X_val = self.x.iloc[val_idx]
            y_train = self.y.iloc[train_idx]
            y_val = self.y.iloc[val_idx]
            yield X_train, X_val, y_train, y_val


In [95]:
    def fit(self, key):
        if key == 'xgb':
            self._fit_xgb()
        elif key == 'lgb':
            self._fit_lgb()
        elif key == 'cat':
            self._fit_cat()



In [96]:
def _fit_xgb(self):
    import xgboost as xgb
    mae_per_fold = []
    r2_per_fold = []
    self.cv_splitter = self.split_data()

    for fold, (X_train, X_val, y_train, y_val) in enumerate(self.cv_splitter):
        model = xgb.XGBRegressor(**self.params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='mae', early_stopping_rounds=100, verbose=False)
        eval_results = model.evals_result()
        val_mae = eval_results['validation_0']['mae'][-1]
        
        y_val_preds = model.predict(X_val)
        val_r2 = r2_score(y_val, y_val_preds)

        if val_mae < self.best_mae:
            self.best_mae = val_mae
            self.best_model = model

        print(f"FOLD {fold} | MAE: {val_mae:.5f} | R2: {val_r2:.5f}")
        mae_per_fold.append(val_mae)
        r2_per_fold.append(val_r2)

    print(f"Average MAE across all folds: {np.mean(mae_per_fold):.5f}")
    print(f"Average R2 across all folds: {np.mean(r2_per_fold):.5f}")

In [97]:
def _fit_lgb(self):
    import lightgbm as lgb
    mae_per_fold = []
    r2_per_fold = []
    self.cv_splitter = self.split_data()

    for fold, (X_train, X_val, y_train, y_val) in enumerate(self.cv_splitter):
        model = lgb.LGBMRegressor(**self.params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='mae', early_stopping_rounds=100)
        eval_results = model.evals_result_
        val_mae = eval_results['valid_0']['l1'][-1]
        
        y_val_preds = model.predict(X_val)
        val_r2 = r2_score(y_val, y_val_preds)

        if val_mae < self.best_mae:
            self.best_mae = val_mae
            self.best_model = model

        print(f"FOLD {fold} | MAE: {val_mae:.5f} | R2: {val_r2:.5f}")
        mae_per_fold.append(val_mae)
        r2_per_fold.append(val_r2)

    print(f"Average MAE across all folds: {np.mean(mae_per_fold):.5f}")
    print(f"Average R2 across all folds: {np.mean(r2_per_fold):.5f}")

In [98]:
def _fit_cat(self):
    from catboost import CatBoostRegressor
    from sklearn.metrics import mean_absolute_error, r2_score
    rmse_per_fold = []
    mae_per_fold = []
    r2_per_fold = []
    self.cv_splitter = self.split_data()

    for fold, (X_train, X_val, y_train, y_val) in enumerate(self.cv_splitter):
        model = CatBoostRegressor(**self.params, loss_function='MAE')
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        y_pred = model.predict(X_val)

        val_mae = mean_absolute_error(y_val, y_pred)
        val_r2 = r2_score(y_val, y_pred)

        print(f"FOLD {fold} | MAE: {val_mae:.5f} | R²: {val_r2:.5f}")
        mae_per_fold.append(val_mae)
        r2_per_fold.append(val_r2)

        if val_mae < self.best_mae:
            self.best_mae = val_mae
            self.best_model = model

    print(f"Average MAE across all folds: {np.mean(mae_per_fold):.5f}")
    print(f"Average R² across all folds: {np.mean(r2_per_fold):.5f}")

In [99]:
    def predict(self):
        self.cv_splitter = self.split_data()
        all_rmse = []
        all_rmse_train = []
        for fold, (X_train, X_val, y_train, y_val) in enumerate(self.cv_splitter):
            if self.best_model is None:
                raise ValueError("Model has not been trained yet. Call 'fit()' method first.")
            else:
                y_preds = self.best_model.predict(X_val)
                rmse = np.sqrt(mean_squared_error(y_val, y_preds))
                all_rmse.append(rmse)

                train_preds = self.best_model.predict(X_train)
                train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
                all_rmse_train.append(train_rmse)
                print(f"FOLD {fold} | RMSE: {rmse:.5f}")
        avg_rmse = np.mean(all_rmse)
        avg_rmse_train = np.mean(all_rmse_train)
        print(f'Average RMSE Val: {avg_rmse} | Average RMSE Train: {avg_rmse_train}')
        return all_rmse, y_preds, y_val


In [100]:

# Contoh penggunaan
params_bayesian_ridge = {'max_iter': 300}
model_bayesian_ridge = Model(X_train_poly, y, 10, 3, 42, BayesianRidge, params_bayesian_ridge)
model_bayesian_ridge.fit('bayesian_ridge')  # Make sure to call fit() first
all_mae_bayesian, all_r2_bayesian, y_preds_bayesian, y_val_linear = model_bayesian_ridge.predict()

FOLD 0 | MAE: 15709.39332 | R2: 0.99992
FOLD 1 | MAE: 14463.13038 | R2: 0.99997
FOLD 2 | MAE: 14220.83325 | R2: 0.99996
FOLD 3 | MAE: 15177.41071 | R2: 0.99992
FOLD 4 | MAE: 14268.30808 | R2: 0.99995
FOLD 5 | MAE: 15402.17619 | R2: 0.99995
FOLD 6 | MAE: 14259.90251 | R2: 0.99996
FOLD 7 | MAE: 14069.82924 | R2: 0.99958
FOLD 8 | MAE: 14412.10680 | R2: 0.99997
FOLD 9 | MAE: 8356.86415 | R2: 0.99808
FOLD 10 | MAE: 14782.73420 | R2: 0.99996
FOLD 11 | MAE: 15120.67740 | R2: 0.99994
FOLD 12 | MAE: 14962.39004 | R2: 0.99995
FOLD 13 | MAE: 15060.01028 | R2: 0.99995
FOLD 14 | MAE: 14827.07037 | R2: 0.99990
FOLD 15 | MAE: 8510.73842 | R2: 0.99814
FOLD 16 | MAE: 14436.23953 | R2: 0.99996
FOLD 17 | MAE: 13391.62127 | R2: 0.99966
FOLD 18 | MAE: 15066.06966 | R2: 0.99992
FOLD 19 | MAE: 15040.36935 | R2: 0.99993
FOLD 20 | MAE: 15428.73520 | R2: 0.99994
FOLD 21 | MAE: 14183.19816 | R2: 0.99997
FOLD 22 | MAE: 14707.45601 | R2: 0.99997
FOLD 23 | MAE: 14334.72766 | R2: 0.99995
FOLD 24 | MAE: 14294.59593 |

In [101]:
# Parameter untuk Linear Regression
params_linear = {}
model_linear = Model(X_train_poly, y, 10, 3, 42, LinearRegression, params_linear)
model_linear.fit('linear')
all_mae_linear,all_r2_linier, y_preds_linear, y_val_linear = model_linear.predict()

FOLD 0 | MAE: 21436.84538 | R2: 0.99986
FOLD 1 | MAE: 19611.39413 | R2: 0.99995
FOLD 2 | MAE: 21983.70402 | R2: 0.99992
FOLD 3 | MAE: 25277.89157 | R2: 0.99988
FOLD 4 | MAE: 25255.84843 | R2: 0.99987
FOLD 5 | MAE: 25131.79622 | R2: 0.99989
FOLD 6 | MAE: 20330.81605 | R2: 0.99990
FOLD 7 | MAE: 18430.33113 | R2: 0.99962
FOLD 8 | MAE: 20183.87962 | R2: 0.99994
FOLD 9 | MAE: 11358.70754 | R2: 0.99805
FOLD 10 | MAE: 22526.74029 | R2: 0.99990
FOLD 11 | MAE: 21980.36738 | R2: 0.99991
FOLD 12 | MAE: 18964.38740 | R2: 0.99992
FOLD 13 | MAE: 20721.78488 | R2: 0.99992
FOLD 14 | MAE: 22154.34793 | R2: 0.99987
FOLD 15 | MAE: 11317.26125 | R2: 0.99822
FOLD 16 | MAE: 19677.17968 | R2: 0.99990
FOLD 17 | MAE: 18378.00377 | R2: 0.99969
FOLD 18 | MAE: 23684.89001 | R2: 0.99853
FOLD 19 | MAE: 24931.13734 | R2: 0.99987
FOLD 20 | MAE: 21582.17088 | R2: 0.99985
FOLD 21 | MAE: 21430.51372 | R2: 0.99993
FOLD 22 | MAE: 19754.63383 | R2: 0.99994
FOLD 23 | MAE: 20630.60919 | R2: 0.99992
FOLD 24 | MAE: 21539.71237

In [102]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
# Parameter untuk Ridge Regression
params_ridge = {'alpha': 1.0}
model_ridge = Model(X_train_poly, y, 10, 3, 42, Ridge, params_ridge)
model_ridge.fit('ridge')
all_mae_ridge,all_r2_ridge, y_preds_ridge, y_val_ridge = model_ridge.predict()

FOLD 0 | MAE: 17427.18860 | R2: 0.99987
FOLD 1 | MAE: 15808.94336 | R2: 0.99996
FOLD 2 | MAE: 15733.31993 | R2: 0.99995
FOLD 3 | MAE: 16710.26657 | R2: 0.99990
FOLD 4 | MAE: 16401.94523 | R2: 0.99991
FOLD 5 | MAE: 17464.30232 | R2: 0.99992
FOLD 6 | MAE: 16033.01495 | R2: 0.99991
FOLD 7 | MAE: 15755.48280 | R2: 0.99962
FOLD 8 | MAE: 15534.11087 | R2: 0.99995
FOLD 9 | MAE: 9291.15156 | R2: 0.99810
FOLD 10 | MAE: 16879.15275 | R2: 0.99991
FOLD 11 | MAE: 16775.78332 | R2: 0.99993
FOLD 12 | MAE: 16785.28905 | R2: 0.99992
FOLD 13 | MAE: 16393.96689 | R2: 0.99993
FOLD 14 | MAE: 16309.35232 | R2: 0.99989
FOLD 15 | MAE: 9104.20589 | R2: 0.99822
FOLD 16 | MAE: 16120.00191 | R2: 0.99992
FOLD 17 | MAE: 14793.04766 | R2: 0.99969
FOLD 18 | MAE: 17459.82826 | R2: 0.99935
FOLD 19 | MAE: 16961.23777 | R2: 0.99990
FOLD 20 | MAE: 17725.40837 | R2: 0.99986
FOLD 21 | MAE: 15973.44821 | R2: 0.99995
FOLD 22 | MAE: 16226.19090 | R2: 0.99995
FOLD 23 | MAE: 15645.42118 | R2: 0.99994
FOLD 24 | MAE: 16335.55293 |

In [103]:

cb_params = {'random_seed': 4897,  'verbose': False}
model_cat = Model(X_train_poly, y, 10, 3, 42, CatBoostRegressor, cb_params)

# Fit the model using RepeatedKFold
model_cat.fit('cat')
all_mae_cat,all_r2_cat, y_preds, y_val = model_cat.predict()

FOLD 0 | MAE: 87762.01287 | R2: 0.93705
FOLD 1 | MAE: 86880.64361 | R2: 0.94652
FOLD 2 | MAE: 65726.23376 | R2: 0.99078
FOLD 3 | MAE: 99485.06117 | R2: 0.92139
FOLD 4 | MAE: 90698.31103 | R2: 0.94388
FOLD 5 | MAE: 67350.95455 | R2: 0.98751
FOLD 6 | MAE: 77333.56424 | R2: 0.94629
FOLD 7 | MAE: 88239.57002 | R2: 0.96124
FOLD 8 | MAE: 90803.51865 | R2: 0.95650
FOLD 9 | MAE: 85508.29227 | R2: 0.97255
FOLD 10 | MAE: 96109.42094 | R2: 0.93939
FOLD 11 | MAE: 104440.06739 | R2: 0.92176
FOLD 12 | MAE: 89628.85121 | R2: 0.95871
FOLD 13 | MAE: 71356.10934 | R2: 0.98949
FOLD 14 | MAE: 63705.63720 | R2: 0.98884
FOLD 15 | MAE: 97205.89445 | R2: 0.89492
FOLD 16 | MAE: 67581.59183 | R2: 0.98402
FOLD 17 | MAE: 91508.13390 | R2: 0.96195
FOLD 18 | MAE: 80742.08958 | R2: 0.98009
FOLD 19 | MAE: 71299.76345 | R2: 0.97543
FOLD 20 | MAE: 101523.35677 | R2: 0.94693
FOLD 21 | MAE: 84035.67959 | R2: 0.96240
FOLD 22 | MAE: 97565.01772 | R2: 0.92435
FOLD 23 | MAE: 72072.68244 | R2: 0.97893
FOLD 24 | MAE: 82209.946

In [104]:
params = {'random_state':42}


model_xgb = Model(X_train_poly, y, 10, 3, 42, XGBRegressor, params)

# Fit the model using RepeatedKFold
model_xgb.fit('xgb')
all_mae_xgb,all_r2_xgb, y_preds, y_val = model_xgb.predict()

FOLD 0 | MAE: 49187.08262 | R2: 0.97625
FOLD 1 | MAE: 38578.98495 | R2: 0.99282
FOLD 2 | MAE: 38416.31022 | R2: 0.98616
FOLD 3 | MAE: 49011.18986 | R2: 0.97215
FOLD 4 | MAE: 44959.10600 | R2: 0.98928
FOLD 5 | MAE: 36397.74845 | R2: 0.99590
FOLD 6 | MAE: 32920.53215 | R2: 0.99687
FOLD 7 | MAE: 41330.71661 | R2: 0.99356
FOLD 8 | MAE: 45285.32986 | R2: 0.98656
FOLD 9 | MAE: 43027.11259 | R2: 0.98883
FOLD 10 | MAE: 39802.70627 | R2: 0.99359
FOLD 11 | MAE: 39251.79278 | R2: 0.99323
FOLD 12 | MAE: 57915.07621 | R2: 0.95555
FOLD 13 | MAE: 37614.23215 | R2: 0.99249
FOLD 14 | MAE: 32453.28973 | R2: 0.99803
FOLD 15 | MAE: 44642.14782 | R2: 0.97570
FOLD 16 | MAE: 37498.81784 | R2: 0.99345
FOLD 17 | MAE: 47479.50409 | R2: 0.98556
FOLD 18 | MAE: 43195.32594 | R2: 0.98888
FOLD 19 | MAE: 38716.29443 | R2: 0.99202
FOLD 20 | MAE: 38495.37352 | R2: 0.99591
FOLD 21 | MAE: 41125.98351 | R2: 0.98974
FOLD 22 | MAE: 52636.27275 | R2: 0.97728
FOLD 23 | MAE: 31212.06918 | R2: 0.99672
FOLD 24 | MAE: 41005.70311

In [105]:
params = {'random_state':42,'verbosity':-1}
model_lgbm = Model(X_train_poly, y, 10, 3, 42, lgbm.LGBMRegressor, params)

# Fit the model using RepeatedKFold
model_lgbm.fit('lgb')
all_mae_lgb,all_r2_lgb, y_preds, y_val = model_lgbm.predict()

FOLD 0 | MAE: 53276.40240 | R2: 0.97307
FOLD 1 | MAE: 39642.11472 | R2: 0.99180
FOLD 2 | MAE: 39823.62088 | R2: 0.99253
FOLD 3 | MAE: 54541.67407 | R2: 0.96693
FOLD 4 | MAE: 48306.22062 | R2: 0.98250
FOLD 5 | MAE: 38750.28448 | R2: 0.99676
FOLD 6 | MAE: 41134.37548 | R2: 0.98999
FOLD 7 | MAE: 37631.79641 | R2: 0.99510
FOLD 8 | MAE: 46904.41899 | R2: 0.98745
FOLD 9 | MAE: 40770.53448 | R2: 0.99277
FOLD 10 | MAE: 51182.78039 | R2: 0.98443
FOLD 11 | MAE: 43593.39582 | R2: 0.98595
FOLD 12 | MAE: 55366.24178 | R2: 0.97276
FOLD 13 | MAE: 38197.81986 | R2: 0.99476
FOLD 14 | MAE: 36560.14634 | R2: 0.99516
FOLD 15 | MAE: 48676.55450 | R2: 0.97238
FOLD 16 | MAE: 35000.80862 | R2: 0.99740
FOLD 17 | MAE: 55808.68692 | R2: 0.98208
FOLD 18 | MAE: 49839.36969 | R2: 0.99167
FOLD 19 | MAE: 35722.81733 | R2: 0.99322
FOLD 20 | MAE: 47345.85351 | R2: 0.98466
FOLD 21 | MAE: 51264.45037 | R2: 0.98538
FOLD 22 | MAE: 53797.37520 | R2: 0.98131
FOLD 23 | MAE: 36018.79946 | R2: 0.99254
FOLD 24 | MAE: 39197.70662